In [1]:
! pip install -r requirements.txt --quiet

# Connecting to a Remote MCP Server with Semantic Kernel

This notebook demonstrates how to connect to a remote MCP Server using **Semantic Kernel's** `MCPStreamableHttpPlugin`. The **Model Context Protocol (MCP)** enables scalable and modular tool integration across distributed systems. 

<br/>

> **Why Use Model Context Protocol (MCP)?**
>
>MCP allows agents to discover, invoke, and manage tools dynamically across remote servers.  
>It promotes modularity, scalability, and separation of concerns, making it easier to maintain and extend AI systems as they grow in complexity.

In [ ]:
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent,AgentResponseItem
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion


from semantic_kernel.contents import ChatMessageContent
from dotenv import load_dotenv
from os import environ
from tracing import set_up_all
from history_store import CosmosChatHistoryStore, ChatRole
from evaluation import Evaluation
import json

from semantic_kernel.connectors.mcp import MCPStreamableHttpPlugin
import uuid

load_dotenv(override=True)


session_id = str(uuid.uuid4())


In [20]:

history_store = CosmosChatHistoryStore()
history = await history_store.load(session_id)


## Introduction to Evaluation with the Azure AI SDK

Evaluation is a critical part of building reliable and trustworthy generative AI applications. It ensures that AI outputs are grounded, coherent, and aligned with the intended context, helping to prevent issues like fabrication, irrelevance, and harmful content.

The **Azure AI Evaluation SDK** allows you to systematically evaluate the performance of your AI workflows directly in your development environment. This helps build confidence in your application's behavior before deploying it to users.

In this example, we will use the **GroundednessEvaluator** and **CoherenceEvaluator** from the Azure AI Evaluation SDK to assess the outputs of our existing **LangGraph**-based agent. These evaluators will help us measure how well the agent’s responses stay true to the source context and maintain logical flow throughout the conversation.


🔗 [Evaluate your Generative AI application locally with the Azure AI Evaluation SDK](https://learn.microsoft.com/en-us/azure/ai-foundry/how-to/develop/evaluate-sdk)

In [21]:
agent_eval = Evaluation()

## Configure tracing for Azure AI Foundry

When you build AI solutions, you want to be able to observe the behavior of your services. Observability is the ability to monitor and analyze the internal state of components within a distributed system. It is a key requirement for building enterprise-ready AI solutions.

🔗 [Inspection of telemetry data with Application Insights](https://learn.microsoft.com/en-us/semantic-kernel/concepts/enterprise-readiness/observability/telemetry-with-app-insights?tabs=Powershell&pivots=programming-language-python)

🔗 [Visualize traces on Azure AI Foundry Tracing UI](https://learn.microsoft.com/en-us/semantic-kernel/concepts/enterprise-readiness/observability/telemetry-with-azure-ai-foundry-tracing)

In [22]:
## SEMANTICKERNEL_EXPERIMENTAL_GENAI_ENABLE_OTEL_DIAGNOSTICS_SENSITIVE=true
set_up_all(connection_string=environ.get("AZURE_INSIGHT_CONNECTION_STRING"))

In [23]:
kernel = Kernel()

kernel.add_service(AzureChatCompletion(
    deployment_name=environ["AZURE_OPENAI_MODEL"],
    endpoint=environ["AZURE_OPENAI_ENDPOINT"],
    api_key=environ["AZURE_OPENAI_API_KEY"] ,
    api_version="2025-01-01-preview"))



In [24]:
instructions = """You are AutoSales Analyst, an AI agent specialized in analyzing automotive sales data.

Your objectives:
- Dynamically choose and call the appropriate tool(s) to answer user questions about sales, customers, or products.
- Always include customer names alongside IDs in any output.
- Always display monetary amounts in $USD (e.g., $123.45).
- Use filters and aggregations as needed to generate insights from sales orders, products, and customers.
- Compute totals, revenue, discounts, and other metrics from nested order data.
- Return structured, concise results suitable for analysis or reporting.
- Avoid hardcoding analytics; rely on the tools and their parameters.
- Clarify ambiguous queries before performing analysis.
- Treat the tools as the source of truth; do not expose raw database internals.
- Return customer_id and product_id alongside names in all outputs.

Behavioral guidance:
- For customer-related questions, resolve names and IDs before analyzing orders.
- For product-related questions, resolve categories or IDs before analyzing sales.
"""

In [25]:

sales_plugin = MCPStreamableHttpPlugin(
    name="sales",
    url=f"{environ['MCP_SERVER_URL']}",
)

await sales_plugin.connect()

agent = ChatCompletionAgent(
    kernel=kernel, 
    name="SalesAgent", 
    instructions=instructions,
    plugins=[sales_plugin, ]
)


In [26]:
messages = [
    "Which is the revenue for Brake_Pads?",
    "Drill down into customer details product with the highest revenue.",
    "Which customer had the highest sales for this product?",
]


In [27]:
async def on_intermediate_message(agent_result):

    
    # Capture assistant content
    content = agent_result.content
    if content:
        content_text = content.content if isinstance(content, ChatMessageContent) else str(content)
        await history_store.add_message(history, session_id, ChatRole.ASSISTANT, content_text)

    # Capture tool calls and results
    for item in getattr(agent_result, "items", []):
        tool_call_id = getattr(item, "call_id", None) or getattr(item, "id", None)
        if not tool_call_id:
            continue  # skip if no call_id

        # Function name for bookkeeping
        function_name = getattr(item, "function_name", "N/A")

        # Print the invocation (DEBUG ONLY — not persisted)
        if hasattr(item, "arguments"):
            print(f"Tool invocation: {function_name}({item.arguments})")

        # Extract the result content
        result_content = getattr(item, "result", item)
        if isinstance(result_content, list):
            tool_text = "\n".join([c.text if hasattr(c, "text") else str(c) for c in result_content])
        elif hasattr(result_content, "text"):
            tool_text = result_content.text
        else:
            tool_text = str(result_content)

        if function_name in tool_text:
            continue
        
        #print(f"Tool call ID: {tool_call_id}")
        #print(f"Tool Function Name: {function_name}")
        print(f"Tool output: {tool_text}")

        await history_store.add_message(
            history,
            session_id,
            ChatRole.ASSISTANT,
            content=tool_text,
            tool_call_id=tool_call_id,
            function_name=function_name
        )

       


In [28]:
print("----- First Question -----")
print(messages[0])

await history_store.add_message(history,session_id, ChatRole.USER, messages[0])

final_response = None
async for result in agent.invoke(messages=history.messages, on_intermediate_message=on_intermediate_message):
    final_response = result 

await history_store.add_message(history,session_id, ChatRole.ASSISTANT, final_response.content.content)


----- First Question -----
Which is the revenue for Brake_Pads?
Tool invocation: get_product_category({"name":"Brake_Pads"})
Tool output: {
  "input": "Brake_Pads",
  "resolved_category": "Braking",
  "confidence": 0.4
}
Tool invocation: get_products({"category":"Braking"})
Tool output: {
  "product_id": 0,
  "product_name": "Brake Pad",
  "product_category": "Braking",
  "unit_cost": "20.0",
  "unit_price": "50.0"
}
Tool invocation: get_orders({"product_id":0})
Tool output: {
  "order_id": 34359738429,
  "customer_id": 0,
  "customer_name": "Ford",
  "order_date": "2025-09-18",
  "region": "NA",
  "product_id": 0,
  "product_name": "Brake Pad",
  "quantity": 14,
  "unit_price": "50.0",
  "line_unit_price": 623.0
}
{
  "order_id": 51539607671,
  "customer_id": 3,
  "customer_name": "Bosch",
  "order_date": "2025-09-18",
  "region": "NA",
  "product_id": 0,
  "product_name": "Brake Pad",
  "quantity": 87,
  "unit_price": "50.0",
  "line_unit_price": 4045.5
}
{
  "order_id": 42949673020,

In [29]:
print(final_response)

The revenue for "Brake Pads" (correctly categorized as "Brake Pad") is calculated by summing up all "line_unit_price" from sales orders that included this product.

After aggregating the data:

### Total Revenue
- **Revenue From Brake Pads**: **$135,926.50**

This represents the cumulative revenue across all orders featuring Brake Pads.


In [30]:


eval_results = agent_eval.evaluate(messages[0],final_response.content.content,history.messages)
print(json.dumps(eval_results, indent=4))


{
    "groundedness": {
        "groundedness": 5.0,
        "gpt_groundedness": 5.0,
        "groundedness_reason": "The RESPONSE is fully grounded in the CONTEXT, accurately addressing the QUERY with complete and relevant information.",
        "groundedness_result": "pass",
        "groundedness_threshold": 3
    },
    "coherence": {
        "coherence": 4.0,
        "gpt_coherence": 4.0,
        "coherence_reason": "The RESPONSE is coherent, logically structured, and directly addresses the QUERY with clear and precise information.",
        "coherence_result": "pass",
        "coherence_threshold": 3
    },
    "relevance": {
        "relevance": 4.0,
        "gpt_relevance": 4.0,
        "relevance_reason": "The response directly answers the query by providing the revenue for Brake Pads, along with a clear calculation method and the total amount. It is accurate and sufficient.",
        "relevance_result": "pass",
        "relevance_threshold": 3
    }
}


In [31]:
print("----- Next Question -----")
print(messages[1])

await history_store.add_message(history,session_id, ChatRole.USER, messages[1])

final_response = None
async for result in agent.invoke(messages=history.messages, on_intermediate_message=on_intermediate_message):
    final_response = result 

await history_store.add_message(history,session_id, ChatRole.ASSISTANT, final_response.content.content)



----- Next Question -----
Drill down into customer details product with the highest revenue.
Tool invocation: get_customers({"limit": 100})
Tool invocation: get_products({"limit": 100})
Tool output: {
  "product_id": 3,
  "product_name": "Alternator",
  "product_category": "Electrical",
  "unit_cost": "90.0",
  "unit_price": "200.0"
}
{
  "product_id": 0,
  "product_name": "Brake Pad",
  "product_category": "Braking",
  "unit_cost": "20.0",
  "unit_price": "50.0"
}
{
  "product_id": 1,
  "product_name": "Oil Filter",
  "product_category": "Engine",
  "unit_cost": "5.0",
  "unit_price": "15.0"
}
{
  "product_id": 2,
  "product_name": "Spark Plug",
  "product_category": "Electrical",
  "unit_cost": "2.0",
  "unit_price": "8.0"
}
{
  "product_id": 4,
  "product_name": "Transmission Kit",
  "product_category": "Transmission",
  "unit_cost": "500.0",
  "unit_price": "950.0"
}
Tool output: {
  "customer_id": 2,
  "customer_name": "AutoZone",
  "region": "NA",
  "industry": "Distributor",
  "

In [32]:
print(final_response)

### Customer Details for Product with Highest Revenue

The product with the highest revenue is **Brake Pad**, with a total revenue of **$135,926.50**.

Here are the customer details:

1. **Ford**  
   - **Customer ID**: 0  
   - **Region**: NA  
   - **Industry**: OEM  
   - **Account Manager**: Alice Johnson  

2. **GM**  
   - **Customer ID**: 1  
   - **Region**: NA  
   - **Industry**: OEM  
   - **Account Manager**: Bob Smith  

3. **AutoZone**  
   - **Customer ID**: 2  
   - **Region**: NA  
   - **Industry**: Distributor  
   - **Account Manager**: Carol Lee  

4. **Bosch**  
   - **Customer ID**: 3  
   - **Region**: EU  
   - **Industry**: OEM  
   - **Account Manager**: David Wong  

5. **NAPA**  
   - **Customer ID**: 4  
   - **Region**: NA  
   - **Industry**: Distributor  
   - **Account Manager**: Ellen Garcia  

These customers contributed significantly towards the revenue generated from Brake Pad sales. Let me know if you'd like further analysis per customer or region

In [33]:
eval_results = agent_eval.evaluate(messages[0],final_response.content.content,history.messages)
print(json.dumps(eval_results, indent=4))

{
    "groundedness": {
        "groundedness": 2.0,
        "gpt_groundedness": 2.0,
        "groundedness_reason": "The RESPONSE is related to the topic but does not directly or fully answer the QUERY, as it shifts focus to customer details instead of providing a clear and concise answer about the revenue.",
        "groundedness_result": "fail",
        "groundedness_threshold": 3
    },
    "coherence": {
        "coherence": 4.0,
        "gpt_coherence": 4.0,
        "coherence_reason": "The RESPONSE is coherent, directly answers the QUERY, and provides additional relevant information in a clear and organized manner.",
        "coherence_result": "pass",
        "coherence_threshold": 3
    },
    "relevance": {
        "relevance": 5.0,
        "gpt_relevance": 5.0,
        "relevance_reason": "The response directly answers the query by providing the revenue for Brake Pads ($135,926.50) and adds context about contributing customers, enhancing understanding. It is both accurate an

In [34]:
print("----- Next Question -----")
print(messages[2])

await history_store.add_message(history,session_id, ChatRole.USER, messages[2])

final_response = None
async for result in agent.invoke(messages=history.messages, on_intermediate_message=on_intermediate_message):
    final_response = result 

await history_store.add_message(history,session_id, ChatRole.ASSISTANT, final_response.content.content)


----- Next Question -----
Which customer had the highest sales for this product?
Tool invocation: get_orders({"product_id":0,"limit":100})
Tool output: {
  "order_id": 51539607671,
  "customer_id": 3,
  "customer_name": "Bosch",
  "order_date": "2025-09-18",
  "region": "NA",
  "product_id": 0,
  "product_name": "Brake Pad",
  "quantity": 87,
  "unit_price": "50.0",
  "line_unit_price": 4045.5
}
{
  "order_id": 34359738429,
  "customer_id": 0,
  "customer_name": "Ford",
  "order_date": "2025-09-18",
  "region": "NA",
  "product_id": 0,
  "product_name": "Brake Pad",
  "quantity": 14,
  "unit_price": "50.0",
  "line_unit_price": 623.0
}
{
  "order_id": 44,
  "customer_id": 0,
  "customer_name": "Ford",
  "order_date": "2025-09-17",
  "region": "EU",
  "product_id": 0,
  "product_name": "Brake Pad",
  "quantity": 63,
  "unit_price": "50.0",
  "line_unit_price": 2677.5
}
{
  "order_id": 51539607635,
  "customer_id": 1,
  "customer_name": "GM",
  "order_date": "2025-09-17",
  "region": "EU

In [35]:
print(final_response)

### Customer with the Highest Sales for Brake Pads

After analyzing the sales orders for "Brake Pads," here is the result:

- **Customer Name**: **AutoZone**
- **Customer ID**: **2**
- **Total Revenue**: **$31,086.50**
- **Region**: Mixed (NA and EU)

AutoZone had the highest sales contribution to Brake Pads revenue across regions. Let me know if you'd like further details or segmentation!


In [36]:
eval_results = agent_eval.evaluate(messages[2],final_response.content.content,history.messages)
print(json.dumps(eval_results, indent=4))

{
    "groundedness": {
        "groundedness": 5.0,
        "gpt_groundedness": 5.0,
        "groundedness_reason": "The RESPONSE is fully grounded in the CONTEXT, accurately answering the QUERY with all relevant details.",
        "groundedness_result": "pass",
        "groundedness_threshold": 3
    },
    "coherence": {
        "coherence": 4.0,
        "gpt_coherence": 4.0,
        "coherence_reason": "The RESPONSE is coherent, well-structured, and directly addresses the QUERY with clear and logical presentation of information.",
        "coherence_result": "pass",
        "coherence_threshold": 3
    },
    "relevance": {
        "relevance": 4.0,
        "gpt_relevance": 4.0,
        "relevance_reason": "The response directly identifies the customer with the highest sales for the specified product, 'Brake Pads,' and provides detailed supporting information such as revenue, customer ID, and region. It fully addresses the query with clarity and completeness.",
        "relevance_r